<a href="https://colab.research.google.com/github/jpiche0720/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# DIRECTIONS

italicized text# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [X] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [X] Do one-hot encoding of categorical features.
- [X] Do feature selection with `SelectKBest`.
- [X] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [X] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


#Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# BEGIN CODE

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

# Subset Data 

- for single family dwellings 100k - 2m
- train jan-march for april testing


In [0]:
print(df.shape)
df.head()

In [0]:
df.describe()

In [0]:
df['BUILDING_CLASS_CATEGORY'].value_counts()

In [0]:
data = df[df['BUILDING_CLASS_CATEGORY']== '01 ONE FAMILY DWELLINGS']
print(data.shape)
data.head()

In [8]:
data = data[data['SALE_PRICE']>= 100000]
data = data[data['SALE_PRICE']<= 2000000]
data['SALE_PRICE'].max()

2000000

In [0]:
data['SALE_DATE'] = pd.to_datetime(data['SALE_DATE'])

In [10]:
data['SALE_DATE'].max()

Timestamp('2019-04-30 00:00:00')

In [0]:
data = data.dropna(axis=1)

# Train/Test

In [0]:
train = data[data['SALE_DATE']> pd.to_datetime('04/01/2019')]
test = data[data['SALE_DATE']>= pd.to_datetime('04/01/2019')]

In [0]:
print(train.shape)
train.head()

In [0]:
print(test.shape)
test.head()

# Layout Variables 

In [0]:
target = 'SALE_PRICE'
features = train.columns.drop(['SALE_PRICE','SALE_DATE','TAX_CLASS_AT_TIME_OF_SALE','BUILDING_CLASS_CATEGORY'])
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_true = test[target]

# One-Hot encoding for cat variables 



In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)



In [30]:
X_train.head()

,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_A3,ADDRESS_607 OAK TERRACE,ADDRESS_1642 68TH STREET,ADDRESS_1448 81ST STREET,ADDRESS_2374 EAST 71ST STREET,ADDRESS_2129 MERMAID AVENUE,ADDRESS_427 NEW LOTS AVENUE,ADDRESS_717 RODER AVENUE,ADDRESS_5301 AVENUE M,ADDRESS_3706 BAY VIEW AVENUE,ADDRESS_201-16 26TH AVENUE,ADDRESS_121-18 7TH AVENUE,ADDRESS_317 WARWICK AVENUE,ADDRESS_59-51 PARSONS BLVD,ADDRESS_80-32 190 STREET,...,"LAND_SQUARE_FEET_1,643","LAND_SQUARE_FEET_2,162","LAND_SQUARE_FEET_2,425","LAND_SQUARE_FEET_2,080","LAND_SQUARE_FEET_2,523","LAND_SQUARE_FEET_2,156","LAND_SQUARE_FEET_1,320","LAND_SQUARE_FEET_1,830","LAND_SQUARE_FEET_3,125","LAND_SQUARE_FEET_2,730","LAND_SQUARE_FEET_8,530","LAND_SQUARE_FEET_5,820","LAND_SQUARE_FEET_3,733","LAND_SQUARE_FEET_5,040","LAND_SQUARE_FEET_4,100","LAND_SQUARE_FEET_1,470","LAND_SQUARE_FEET_3,003","LAND_SQUARE_FEET_1,832","LAND_SQUARE_FEET_2,435","LAND_SQUARE_FEET_3,570","LAND_SQUARE_FEET_1,536","LAND_SQUARE_FEET_2,721","LAND_SQUARE_FEET_3,880","LAND_SQUARE_FEET_2,352","LAND_SQUARE_FEET_2,340","LAND_SQUARE_FEET_4,360","LAND_SQUARE_FEET_3,142","LAND_SQUARE_FEET_5,036",GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_A3
18500,1,0,0,0,0,1,0,0,0,0,0,0,1,0,2555,25,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1104.0,2000.0,1,0,0,0,0,0,0,0,0,0
18520,0,1,0,0,0,1,0,0,0,0,0,0,1,0,5575,27,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3690.0,1920.0,0,1,0,0,0,0,0,0,0,0
18521,0,1,0,0,0,1,0,0,0,0,0,0,1,0,6293,29,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1296.0,1930.0,0,0,1,0,0,0,0,0,0,0
18525,0,1,0,0,0,1,0,0,0,0,0,0,1,0,8448,68,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1665.0,1970.0,1,0,0,0,0,0,0,0,0,0
18549,0,1,0,0,0,1,0,0,0,0,0,0,1,0,7017,110,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1344.0,1993.0,1,0,0,0,0,0,0,0,0,0


In [31]:
X_test.head()

,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_A3,ADDRESS_607 OAK TERRACE,ADDRESS_1642 68TH STREET,ADDRESS_1448 81ST STREET,ADDRESS_2374 EAST 71ST STREET,ADDRESS_2129 MERMAID AVENUE,ADDRESS_427 NEW LOTS AVENUE,ADDRESS_717 RODER AVENUE,ADDRESS_5301 AVENUE M,ADDRESS_3706 BAY VIEW AVENUE,ADDRESS_201-16 26TH AVENUE,ADDRESS_121-18 7TH AVENUE,ADDRESS_317 WARWICK AVENUE,ADDRESS_59-51 PARSONS BLVD,ADDRESS_80-32 190 STREET,...,"LAND_SQUARE_FEET_1,643","LAND_SQUARE_FEET_2,162","LAND_SQUARE_FEET_2,425","LAND_SQUARE_FEET_2,080","LAND_SQUARE_FEET_2,523","LAND_SQUARE_FEET_2,156","LAND_SQUARE_FEET_1,320","LAND_SQUARE_FEET_1,830","LAND_SQUARE_FEET_3,125","LAND_SQUARE_FEET_2,730","LAND_SQUARE_FEET_8,530","LAND_SQUARE_FEET_5,820","LAND_SQUARE_FEET_3,733","LAND_SQUARE_FEET_5,040","LAND_SQUARE_FEET_4,100","LAND_SQUARE_FEET_1,470","LAND_SQUARE_FEET_3,003","LAND_SQUARE_FEET_1,832","LAND_SQUARE_FEET_2,435","LAND_SQUARE_FEET_3,570","LAND_SQUARE_FEET_1,536","LAND_SQUARE_FEET_2,721","LAND_SQUARE_FEET_3,880","LAND_SQUARE_FEET_2,352","LAND_SQUARE_FEET_2,340","LAND_SQUARE_FEET_4,360","LAND_SQUARE_FEET_3,142","LAND_SQUARE_FEET_5,036",GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_A3
18235,1,0,0,0,0,1,0,0,0,0,0,0,1,0,5913,878,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2272.0,1930.0,0,0,1,0,0,0,0,0,0,0
18239,1,0,0,0,0,1,0,0,0,0,0,0,1,0,5488,48,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,720.0,1935.0,0,0,0,0,0,1,0,0,0,0
18244,0,1,0,0,0,1,0,0,0,0,0,0,1,0,5936,31,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2210.0,1925.0,0,0,1,0,0,0,0,0,0,0
18280,0,1,0,0,0,1,0,0,0,0,0,0,1,0,7813,24,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1520.0,1915.0,1,0,0,0,0,0,0,0,0,0
18285,0,1,0,0,0,1,0,0,0,0,0,0,1,0,8831,160,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,840.0,1925.0,0,1,0,0,0,0,0,0,0,0


# SELECTKBEST


In [0]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
import numpy as np

selector = SelectKBest(score_func=f_regression,k=12)

In [33]:


X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

((606, 12), (647, 12))

In [0]:


colnames = X_train.columns
selected_mask = selector.get_support()
col_sel = colnames[selected_mask]
col_unsel = colnames[~selected_mask]

In [35]:
for name in col_sel:
  print(name)

BOROUGH_3
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_BOROUGH PARK
BLOCK
BUILDING_CLASS_AT_PRESENT_A3
ADDRESS_150-57 6TH AVENUE
ADDRESS_110-12 68TH ROAD
ADDRESS_296 STRATFORD ROAD
ADDRESS_240 11TH STREET
LAND_SQUARE_FEET_6,750
GROSS_SQUARE_FEET
BUILDING_CLASS_AT_TIME_OF_SALE_A3


In [36]:

for k in range(1,15):
  
  selector = SelectKBest(score_func=f_regression,k=k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)
  
  model = LinearRegression()
  model.fit(X_train_selected,y_train)
  y_pred = model.predict(X_test_selected)
  
  mae = mean_absolute_error(y_true,y_pred)
  
  print(f'The mae = ${mae:.0f}') 

The mae = $187995
The mae = $182931
The mae = $177827
The mae = $175967
The mae = $175967
The mae = $173898
The mae = $171751
The mae = $171563
The mae = $169783
The mae = $169783
The mae = $168148
The mae = $167411
The mae = $167155
The mae = $167190


# Ridge Regression

In [0]:
model = Ridge(normalize=True)
model.fit(X_train_selected,y_train)
y_pred = model.predict(X_test_selected)


In [44]:
mae = mean_absolute_error(y_true, y_pred)
print(f'The MAE = ${mae:.0f}')

The MAE = $178698
